In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import os

import helper_functions as hf 

path = '/home/ts809/Documents/Code/github/building_resilience/top_to_bottom/' 

In [2]:
## read EXTRAdata files:

# country codes for countries written in the UN format
CCs = pd.read_csv('data/help_data/UN_country_code.csv')
CCs = CCs.dropna(axis = 0).reset_index(drop=True)

# country codes for countries written in the WB format
WBs = pd.read_csv('data/help_data/WB_country_code.csv')
WBs = WBs.dropna(axis = 0).reset_index(drop=True)

#read WB data
WB_data = pd.read_csv('data/WB_WDI_data/wb_data.csv')

#read UN data
UN_data = pd.read_csv('data/UN_data/WPP2019_TotalPopulationBySex.csv')



In [3]:
UN_prediction = hf.choose_UN_scenario(UN_data,'Medium')

Medium scenario population prediction data succesfully extracted in your variable!


In [4]:
WB_2014 = hf.get_year_of_WB_data(2014,WB_data)

WB data for the year 2014 was succesfully imported!


In [6]:
year = 2015
path = 'data/IEA_data/IEA_percentage_breakdown_'+str(year)+'.csv'

# read IEA data
IEA_2015 = pd.read_csv(path)
IEA_2015 = IEA_2015.drop(['split','new_Data'], axis=1)

In [35]:
WB_IEA = hf.merge_WB_and_IEA(WB_2014,IEA_2015)
## WorldBank has inputed thier NaN values as '..', so we need to remove them
WB_IEA = WB_IEA.replace(to_replace='..', value=np.nan)
WB_IEA = WB_IEA.dropna(axis=0).reset_index(drop=True)

In [37]:
industry_elec_list = []
residential_elec_list = []
commercial_elec_list = []

for i in range(len(WB_IEA)):
    df = WB_IEA.iloc[i,:]
    ind_elec = float(df.iloc[3])*float(df.iloc[5])*float(df.iloc[2])*float(df.iloc[6])
    industry_elec_list.append(ind_elec)
    res_elec = float(df.iloc[3])*float(df.iloc[5])*float(df.iloc[2])*float(df.iloc[7])
    residential_elec_list.append(res_elec)
    com_elec = float(df.iloc[3])*float(df.iloc[5])*float(df.iloc[2])*float(df.iloc[8])
    commercial_elec_list.append(com_elec)
    

WB_IEA['Total industry electricity (kWh)'] = industry_elec_list
WB_IEA['Total residential electricity (kWh)'] = residential_elec_list 
WB_IEA['Total commercial electricity (kWh)'] = commercial_elec_list

In [38]:
WB_IEA

,Country Name,Country Code,Access to electricity (% of population),Electric power consumption (kWh per capita),Population growth (annual %),"Population, total",per_industry,per_residential,per_commercial,Total industry electricity (kWh),Total residential electricity (kWh),Total commercial electricity (kWh)
0,Albania,ALB,100,2309.36650255581,-0.207046999760594,2889104,0.211045,0.540434,0.232742,1.408095e+11,3.605775e+11,1.552852e+11
1,Algeria,DZA,99.8770523071289,1362.87191909646,2.03359245164506,38923687,0.347786,0.392070,0.063993,1.842666e+12,2.077298e+12,3.390505e+11
2,Angola,AGO,32,312.228824978484,3.49749274416236,26941779,0.336620,0.663380,0.000000,9.061264e+10,1.785714e+11,0.000000e+00
3,Argentina,ARG,100,3074.70207056563,1.09946109101833,42669500,0.406954,0.328169,0.252993,5.339077e+12,4.305446e+12,3.319166e+12
4,Armenia,ARM,100,1961.61039526467,0.510122729556191,2912403,0.294372,0.348485,0.186147,1.681749e+11,1.990894e+11,1.063459e+11
...,...,...,...,...,...,...,...,...,...,...,...,...
129,"Venezuela, RB",VEN,99.5427474975586,2718.94277456043,0.874378499265389,30045134,0.329587,0.352346,0.311323,2.680118e+12,2.865190e+12,2.531603e+12
130,Vietnam,VNM,99.2,1423.68834535005,1.05347955368319,91714595,0.572030,0.357839,0.053610,7.409419e+12,4.635033e+12,6.943990e+11
131,"Yemen, Rep.",YEM,66.1,219.799922434946,2.65414095594631,25823485,0.036066,0.681967,0.163934,1.353121e+10,2.558629e+11,6.150551e+10
132,Zambia,ZMB,27.9,717.349167873017,3.1213045608998,15399753,0.598985,0.303553,0.062944,1.846139e+11,9.355856e+10,1.940010e+10
